# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [4]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [1]:
import pandas as pd

reviews = pd.read_csv('reviews_sample.csv')
recipes = pd.read_csv('recipes_sample.csv')

In [2]:
import random
recipes = recipes[['id', 'minutes']]
random_recipes = recipes.sample(5)
print(random_recipes)

           id  minutes
18371  379733     1450
24925  273365       25
24700  242772      200
11157  151963       20
18434  323886       75


In [3]:
column1 = max([len(str(random_recipes.loc[i]['id'])) for i in random_recipes.index]) + 4
column2 = len('minutes') + max([len(str(random_recipes.loc[i]['minutes'])) for i in random_recipes.index])

print(f'|{"id": ^{column1}}|{"minutes": ^{column2}}|')
print('|' + '-'*(column1+column2+1) + '|')

for i in random_recipes.index:
    a = random_recipes.loc[i]['id']
    b = random_recipes.loc[i]['minutes']
    print(f'|{a: ^{column1}}|{b: ^{column2}}|')

|    id    |  minutes  |
|----------------------|
|  379733  |   1450    |
|  273365  |    25     |
|  242772  |    200    |
|  151963  |    20     |
|  323886  |    75     |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [4]:
import requests
from bs4 import BeautifulSoup

with open('steps_sample.xml') as f:
    steps_sample = BeautifulSoup(f, 'xml')
    
recipes = pd.read_csv('recipes_sample.csv')

def show_info(id_rec):
    needed_rec = recipes[recipes.id == id_rec]
    name_rec = needed_rec.name.values[0]
    
    steps_dict = {}
    for recipe in steps_sample.find_all('recipe'):
        recipe_id = int(recipe.id.text)
        steps = [step.text for step in recipe.find_all('step')]
        steps_dict[recipe_id] = steps
            
    cont_id = needed_rec.contributor_id.values[0]
    minutes = needed_rec.minutes.values[0]
    
    steps_info = '\n '.join([f"{index+1}: {step}" for index, step in enumerate(steps_dict[id_rec])])

    print(f'{name_rec}\n {steps_info}\n----------\nАвтор: {cont_id}\nСреднее время приготовления: {minutes} минут')

In [5]:
show_info(170895)

leeks and parsnips  sauteed or creamed
 1: clean the leeks and discard the dark green portions
 2: cut the leeks lengthwise then into one-inch pieces
 3: melt the butter in a medium skillet , med
 4: heat
 5: add the garlic and fry 'til fragrant
 6: add leeks and fry until the leeks are tender , about 6-minutes
 7: meanwhile , peel and chunk the parsnips into one-inch pieces
 8: place in a steaming basket and steam 'til they are as tender as you prefer
 9: i like them fork-tender
 10: drain parsnips and add to the skillet with the leeks
 11: add salt and pepper
 12: gently sautee together for 5-minutes
 13: at this point you can serve it , or continue on and cream it:
 14: in a jar with a screw top , add the half-n-half and arrowroot
 15: shake 'til blended
 16: turn heat to low under the leeks and parsnips
 17: pour in the arrowroot mixture , stirring gently as you pour
 18: if too thick , gradually add the water
 19: let simmer for a couple of minutes
 20: taste to adjust seasoning ,

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [6]:
import re

p = re.compile(r'\d+\s(hours|hour|minutes|minute)')

steps_dict = {}
for recipe in steps_sample.find_all('recipe'):
    recipe_id = int(recipe.id.text)
    steps = [step.text for step in recipe.find_all('step')]
    steps_dict[recipe_id] = steps
for step in steps_dict[25082]:
    if bool(p.search(step)):
        print(step)

turn out onto a lightly floured board and knead for about 20 minutes , adding flour as nescessary to keep the dough from sticking to the board
when it has been sufficiently kneaded , cover it with a damp cloth for about 10 minutes and wash and grease the bowl lightly
let the dough rise until it springs back when you stick your finger in it , and it is about twice the size as it was before (this takes about 2 hours
when the dough has risen twice , deflate it again and cover it with the damp cloth again for about 10 minutes , then divide it into and shape it into loaves , buns , etc
bake at 400 for 20 minutes , and then turn the oven down to 350 and bake for 20-30 minutes longer , until the loaf is a lovely brown and sounds hollow when you thump it on the bottom


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [7]:
p = re.compile(r'^this[\w\d\s]+,[ ]?but')

# Заполнить пропущенные значения пустой строкой, чтоб они не мешали работе метода .str.contains
recipes['description'] = recipes['description'].fillna('')

bool_mask = recipes['description'].str.contains(p)
filtered_description = recipes[bool_mask]['description']

len(filtered_description), filtered_description[:3]

(134,
 76     this is a great meal eaten the same day ,but e...
 183    this was adapted from a recipe i found on the ...
 337    this is kind of similar to some of the other v...
 Name: description, dtype: object)

5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [8]:
for step in steps_dict[72367]:
    if re.search(r'\d+\s\/\s\d+', step):
        print(step)
        re.sub(r'(\d+)\s\/\s(\d+)', r'\1\/\2', step)
        print(step)

mix butter , flour , 1 / 3 c
mix butter , flour , 1 / 3 c
sugar and 1-1 / 4 t
sugar and 1-1 / 4 t
mix cream cheese , 1 / 4 c
mix cream cheese , 1 / 4 c
sugar , eggs and 1 / 2 t
sugar , eggs and 1 / 2 t
combine apples , 1 / 3 c
combine apples , 1 / 3 c


### Сегментация текста

In [9]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [10]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from collections import defaultdict
import requests
from bs4 import BeautifulSoup
import pandas as pd

[nltk_data] Downloading package punkt to /Users/test/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/test/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [12]:
with open('steps_sample.xml') as f:
    steps_sample = BeautifulSoup(f, 'xml')
    
id_recipes = defaultdict(list) 
for recipe in steps_sample.find_all('recipe'):
    for steps_list in [step.text for step in recipe.find_all('step')]:
        id_recipes[recipe.find('id').text].append(steps_list)

# Преобразование текстов шагов рецептов в одну строку
all_steps = ' '.join([step for steps_list in id_recipes.values() for step in steps_list])

# Токенизация текста на слова
words = word_tokenize(all_steps)

# Преобразование всех слов в нижний регистр
words = [word.lower() for word in words if word.isalpha()]

# Подсчет уникальных слов
unique_words = set(words)
num_unique_words = len(unique_words)

print("Количество уникальных слов среди всех рецептов:", num_unique_words)

Количество уникальных слов среди всех рецептов: 14926


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [13]:
# Загрузка данных из CSV файла
data = pd.read_csv('recipes_sample.csv')

# Функция для подсчета количества предложений в описании
def count_sentences(description):
    if isinstance(description, str):
        return len(sent_tokenize(description))
    return 0

# Применение функции к столбцу 'description'
data['n_sentences'] = data['description'].apply(count_sentences)

# Сортировка данных по убыванию количества предложений
sorted_data = data.sort_values(by='n_sentences', ascending=False)

# Вывод 5 самых длинных описаний
top_5_longest_descriptions = sorted_data.head(5)

# Вывод результатов
print(top_5_longest_descriptions)


                                                    name      id  minutes  \
18408       my favorite buttercream icing for decorating  334113       30   
481    alligator claws  avocado fritters  with chipot...  287008       45   
22566                          rich barley mushroom soup  328708       60   
6779                                       chocolate tea  205348        6   
16296  little bunny foo foo cake  carrot cake  with c...  316000       68   

       contributor_id   submitted  n_steps  \
18408          681465  2008-10-30     12.0   
481            765354  2008-02-19      NaN   
22566          221776  2008-10-03      NaN   
6779           428824  2007-01-14      NaN   
16296          689540  2008-07-27     14.0   

                                             description  n_ingredients  \
18408  this wonderful icing is used for icing cakes a...            NaN   
481    a translucent golden-brown crust allows the gr...            9.0   
22566  this is one of the best soup

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [14]:
# Загрузка данных из файла CSV
df = pd.read_csv('recipes_sample.csv')

def get_pos_tags_for_recipe(recipe_id):
    recipe_row = df[df['id'] == recipe_id]
    if not recipe_row.empty:
        recipe_name = recipe_row['name'].values[0]
        tokens = word_tokenize(recipe_name) # Название рецепта разбивается на токены (слова)
        tags = pos_tag(tokens) # Для каждого токена определяется часть речи
        tagged_recipe_name = ""
        for word, tag in tags:
            tagged_recipe_name += f"{word} ({tag}) "
        return tagged_recipe_name
    else:
        return "Рецепт с таким ID не найден в базе данных."

# Пример использования
recipe_id = 241106
tagged_recipe_name = get_pos_tags_for_recipe(recipe_id)
print(tagged_recipe_name)

eggplant (JJ) steaks (NNS) with (IN) chickpeas (NNS) feta (VBP) cheese (JJ) and (CC) black (JJ) olives (NNS) 
